# [位置情報を持つデータからKMLファイルを生成してマップ上にプロットするレシピ](https://axross-recipe.com/recipes/264)

In [ ]:
# GC mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# datasetのダウンロード
!wget https://www.opendata.metro.tokyo.lg.jp/fukushihoken/koukyoshisetsu_barrier-free-wc.csv

In [ ]:
# パッケージのインストール
!pip install simplekml

In [ ]:
# datasetの読込
import pandas as pd

df = pd.read_csv('koukyoshisetsu_barrier-free-wc.csv', encoding='cp932')
df

In [ ]:
# SimpleKMLを利用する
import simplekml

# Kmlクラスのインスタンス生成
# 引数はブランクでも実行できるが、識別しやすくするためにnameを設定しましょう
kml = simplekml.Kml(name='Project')

# ポイントの作成
# nameにはポイントの名前
# coordsには経度と緯度を10進法で記述、経度が先なので注意
point = kml.newpoint(name='Tokyo Skytree' ,coords=[(139.8108103, 35.7100069)] ) 

# kmlファイル生成
kml.save("tokyo-skytree.kml")

In [ ]:
# KML2を作成（plekml.Color.rgb(255, 255, 128, 255) ）
# ラベルサイズを2.0に変更（デフォルトは1.0）
point.style.labelstyle.scale = 2.0

# アイコン画像を指定（ここではGoogle Earthの緑ピンを指定）
point.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/pushpin/grn-pushpin.png'

# アイコンサイズを2.0に変更（デフォルトは1.0）
point.style.iconstyle.scale = 2.0

# kmlファイル生成
kml.save("tokyo-skytree_2.kml")

In [ ]:
# tokyotower.kmlを作成（東京タワーの緯度経度、名前を設定してKMLを作成）
point.coords = [(139.7454316, 35.658584)]
point.name = 'Tokyo Tower'
kml.save("tokyotower.kml")

In [ ]:
# kml（拡張データの追加）
#（名称, 値）として引数を入力
point.extendeddata.schemadata.newsimpledata('住所', '東京都港区芝公園４丁目２−８' )
point.extendeddata.schemadata.newsimpledata('高さ', '３３３メートル' )
point.extendeddata.schemadata.newsimpledata('説明', '東京タワーとは東京都港区芝公園にある電波塔の通称である。' )
kml.save("tokyotower_2.kml")

In [ ]:
# kml
# descriptionプロパティに設定
point.description = ('東京タワーとは東京都港区芝公園にある電波塔の通称である。')
kml.save("tokyotower_3.kml")

In [ ]:
# kml
# descriptionにHTMLで東京タワーのサイトURLを記述
point.description = ('<p>東京タワーのサイトURL</p><a href="https://www.tokyotower.co.jp">https://www.tokyotower.co.jp</a>')
kml.save("tokyotower_4.kml")

In [ ]:
# APIの利用
# データの前処理
import requests

#GETリクエスト
data = requests.get('https://api.data.metro.tokyo.lg.jp/v1//MultipurposeToilet?limit=1000')

#レスポンスを表示
print(data.json())

In [ ]:
# 表示の整形
import json

#jsonデータを整形
json_data = json.dumps(data.json(), ensure_ascii=False, indent=4)
print(json_data)

In [ ]:
# API全データの表示
import requests
import pandas as pd
from urllib.parse import quote
from time import sleep

end_cursor = ''
more_results = ''

# 空のDataFrameを作成
df_all = pd.DataFrame()

while more_results != "NO_MORE_RESULTS":
    # end_cursorが空の場合（ループの1回目）
    if not end_cursor:
        data = requests.get('https://api.data.metro.tokyo.lg.jp/v1//MultipurposeToilet?limit=1000').json()

    # end_cursorが空でない場合（ループの2回目以降）
    else:
        data = requests.get(f'https://api.data.metro.tokyo.lg.jp/v1//MultipurposeToilet?limit=1000&cursor={end_cursor_encoded}').json()

    df = pd.json_normalize(data[0]) #JSONが入れ子構造のためnormalizeしてDataFrameに変換
    df_all = pd.concat([df_all, df]) # DataFrameを結合
    end_cursor = data[1]['endCursor'] # endCursorを取得
    end_cursor_encoded = quote(end_cursor) # endCursorをURLエンコード
    more_results = data[1]['moreResults']
    sleep(5)

df_all

In [ ]:
# 欠損値の可視化
import numpy as np

# 正規表現で空白のみのセルをNaN（欠損値）に変換
df_all = df_all.replace(r'^\s*$', np.nan, regex=True)

# 列ごとの欠損値をカウント
df_all.isnull().sum()

In [ ]:
# 緯度または経度が欠損値のレコードを取り除く
df_all  = df_all.dropna(subset=['地理座標.緯度', '地理座標.経度'])

In [ ]:
# 列ごとの欠損値をカウント
df_all.isnull().sum()

In [ ]:
# APIで取得したデータをマップ上にプロット
for index, row in df_all.iterrows():
    print(row['名称.表記'] + ', ' + row['地理座標.緯度'] + ', ' + row['地理座標.経度'])

In [ ]:
# 住所に渋谷区を含むデータを抽出
df_shibuya = df_all[df_all['住所.表記'].str.contains("渋谷区")]
df

In [ ]:
# 抽出データでkmlを作成
import simplekml

#kmlオブジェクトの生成
kml = simplekml.Kml()

#フォルダの作成
floder = kml.newfolder(name='MultipurposeToilets')

# ポイントの生成
for index, row in df_shibuya.iterrows():

  row['名称.表記'] + ', ' + row['地理座標.緯度'] + ', ' + row['地理座標.経度']
  point = floder.newpoint(name=row['名称.表記'], coords=[(row['地理座標.経度'], row['地理座標.緯度'])] )
  point.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/pushpin/ylw-pushpin.png'  #任意の画像
  point.style.labelstyle.color = simplekml.Color.rgb(255, 255,128,255)  #ラベル色を薄い黄色に設定
  point.style.labelstyle.scale = 0.0 # ラベルサイズ
  point.style.iconstyle.scale = 0.7 # アイコンサイズ
  point.extendeddata.schemadata.newsimpledata('住所', row['住所.表記'] ) #拡張データ
  point.extendeddata.schemadata.newsimpledata('設備名称',  row['設備.名称.表記'] )
  point.extendeddata.schemadata.newsimpledata('設置位置',  row['設備.設置位置'] )
  point.extendeddata.schemadata.newsimpledata('性別の区分',  row['設備.ex:性別の区分'] )

# KMLファイルへ出力保存
kml.save('MultipurposeToilets.kml')

In [ ]:
# KMZ（KML圧縮形式）として出力保存
kml.savekmz('MultipurposeToilets.kmz')